In [6]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn

from tools.torch_lib import *

from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import copy
from torchmetrics.regression import MeanAbsolutePercentageError

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
gpu = torch.device('cuda')
cpu = torch.device('cpu')
device = cpu

if torch.cuda.is_available():
    device = gpu
    # The flag below controls whether to allow TF32 on matmul. This flag defaults to False
    # in PyTorch 1.12 and later.
    torch.backends.cuda.matmul.allow_tf32 = True
    # The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
    torch.backends.cudnn.allow_tf32 = True

print(device)

cuda


### Load dataframe

In [9]:
dataset_dir = "dataset/"
dataset_file_name = "1D_2A.csv"
plots_dir = "plots/"

In [10]:
df = pd.read_csv(dataset_dir + dataset_file_name)
df.head()

,ro_well,ro_formation,kanisotrop,A02M01N,A04M01N,A10M01N,A20M05N,A40M05N,A80M10N,rad_well
0,0.01,0.1,1.0,0.110922,0.134909,0.118898,0.105459,0.101749,0.100513,0.040
1,0.01,0.1,1.0,0.107744,0.134384,0.120232,0.105946,0.101906,0.100559,0.042
2,0.01,0.1,1.0,0.104604,0.133653,0.121536,0.106451,0.102069,0.100607,0.044
3,0.01,0.1,1.0,0.101519,0.132742,0.122802,0.106971,0.102238,0.100657,0.046
4,0.01,0.1,1.0,0.098501,0.131673,0.124024,0.107506,0.102412,0.100709,0.048


In [23]:
df

,ro_well,ro_formation,kanisotrop,A04M01N,A10M01N,A20M05N,A40M05N,A80M10N,rad_well
0,0.01000,0.1,1.0,0.134909,0.118898,0.105459,0.101749,0.100513,0.040
1,0.01000,0.1,1.0,0.134384,0.120232,0.105946,0.101906,0.100559,0.042
2,0.01000,0.1,1.0,0.133653,0.121536,0.106451,0.102069,0.100607,0.044
3,0.01000,0.1,1.0,0.132742,0.122802,0.106971,0.102238,0.100657,0.046
4,0.01000,0.1,1.0,0.131673,0.124024,0.107506,0.102412,0.100709,0.048
...,...,...,...,...,...,...,...,...,...
1184113,3.16228,10000.0,5.0,72.021300,377.083000,1556.150000,4787.300000,13859.000000,0.130
1184114,3.16228,10000.0,5.0,62.264400,327.135000,1359.080000,4226.380000,12506.000000,0.140
1184115,3.16228,10000.0,5.0,54.364500,286.489000,1197.160000,3757.950000,11334.100000,0.150
1184116,3.16228,10000.0,5.0,40.133700,212.736000,899.431000,2877.150000,9017.940000,0.175


In [11]:
df.columns

Index(['ro_well', 'ro_formation', 'kanisotrop', 'A02M01N', 'A04M01N',
       'A10M01N', 'A20M05N', 'A40M05N', 'A80M10N', 'rad_well'],
      dtype='object')

In [12]:
# print attribute's min max

In [13]:
for column in df.columns:
    print(f"{column}: min={df[column].min()} max={df[column].max()}")

ro_well: min=0.01 max=3.16228
ro_formation: min=0.1 max=10000.0
kanisotrop: min=1.0 max=5.0
A02M01N: min=0.0201144 max=221.578
A04M01N: min=0.042906 max=700.451
A10M01N: min=0.0910916 max=3299.02
A20M05N: min=0.0945373 max=11120.5
A40M05N: min=0.0976365 max=24778.3
A80M10N: min=0.0991788 max=38051.0
rad_well: min=0.04 max=0.2


In [14]:
print(f"ro_well, ro_formation correlation="
      f"{np.corrcoef(df['ro_well'].to_numpy(), df['ro_formation'].to_numpy())[1][0]}")

ro_well, ro_formation correlation=7.871373887361991e-17


In [15]:
# attributes in logarithmic scale:
for column in df.columns:
    if column == 'd_well':
        continue
    col_data = df[column].to_numpy()
    print(f"{column}: log_min={np.log(col_data.min())} log_max={np.log(col_data.max())} mean={np.mean(col_data)} std={np.std(col_data)}")

ro_well: log_min=-4.605170185988091 log_max=1.1512932864164753 mean=0.5898747846153845 std=0.8332104926178117
ro_formation: log_min=-2.3025850929940455 log_max=9.210340371976184 mean=953.3484185098038 std=2098.6398911467427
kanisotrop: log_min=0.0 log_max=1.6094379124341003 mean=2.183226068969074 std=1.276033218172284
A02M01N: log_min=-3.9063193025114678 log_max=5.4007746719664045 mean=6.371209325846917 std=14.800333932490414
A04M01N: log_min=-3.1487436026878233 log_max=6.551724413294755 mean=17.46536963673781 std=43.020996417288764
A10M01N: log_min=-2.3958896853341227 log_max=8.10138073365337 mean=67.88229960240388 std=179.56774512510788
A20M05N: log_min=-2.3587608133648343 log_max=9.316545530822498 mean=202.81547035922395 std=562.7142969127794
A40M05N: log_min=-2.326503880064628 log_max=10.11772354911712 mean=457.00361950074904 std=1288.8514523369136
A80M10N: log_min=-2.3108309972078964 log_max=10.546682644153423 mean=931.5971180226402 std=2574.9932316569184
rad_well: log_min=-3.2188

In [16]:
attributes_to_drop = ['A02M01N']
df.drop(attributes_to_drop, axis=1, inplace=True)
df.head()

,ro_well,ro_formation,kanisotrop,A04M01N,A10M01N,A20M05N,A40M05N,A80M10N,rad_well
0,0.01,0.1,1.0,0.134909,0.118898,0.105459,0.101749,0.100513,0.040
1,0.01,0.1,1.0,0.134384,0.120232,0.105946,0.101906,0.100559,0.042
2,0.01,0.1,1.0,0.133653,0.121536,0.106451,0.102069,0.100607,0.044
3,0.01,0.1,1.0,0.132742,0.122802,0.106971,0.102238,0.100657,0.046
4,0.01,0.1,1.0,0.131673,0.124024,0.107506,0.102412,0.100709,0.048


### Add dataframe transforms

In [17]:
inputs = np.array(['ro_well', 'ro_formation', 'rad_well', 'kanisotrop'])
outputs = np.array(['A04M01N', 'A10M01N', 'A20M05N', 'A40M05N', 'A80M10N']) # 'A02M01N' dropped

In [18]:
logarithmic_columns = ['ro_formation', 'ro_well']
# normalize data ('min/max' normalization):
interval_th = [-1, 1]     # normalization interval for 'th' activation function
interval_sigmoid = [0, 1] # normalization interval for 'sigmoid' activation function
normalize_interval = interval_th

attributes_transform_dict = {}
df_transformed = df.copy()

# transform output attributes:
for output_attr in outputs:
    attr_transformer = attributes_transform_dict[output_attr] = AttributeTransformer(df_transformed[output_attr].to_numpy())

    # logarithmic transform
    forward, backward = np.log, np.exp
    df_transformed[output_attr] = attr_transformer.transform(forward, backward)
    # scaling transform
    forward, backward = get_standard_scaler_transform(attr_transformer.data)
    df_transformed[output_attr] = attr_transformer.transform(forward, backward)
    # normalize transform
    forward, backward = get_normalize_transforms(attr_transformer.data, normalize_interval)
    df_transformed[output_attr] = attr_transformer.transform(forward, backward)

# logarithm resistance:
for col in logarithmic_columns:
    if col in outputs:
        continue
    df_transformed[col] = df_transformed[col].apply(np.log)

# add normalization
for attribute in df_transformed.columns:
    if attribute in outputs:
        continue
    transform, _ = get_standard_scaler_transform(df_transformed[attribute].to_numpy())
    df_transformed[attribute] = transform(df_transformed[attribute].to_numpy())

    transform, _ = get_normalize_transforms(df_transformed[attribute].to_numpy(), normalize_interval)
    df_transformed[attribute] = transform(df_transformed[attribute].to_numpy())

df_transformed

,ro_well,ro_formation,kanisotrop,A04M01N,A10M01N,A20M05N,A40M05N,A80M10N,rad_well
0,-1.0,-1.0,-1.0,-0.763808,-0.949244,-0.981272,-0.993369,-0.997921,-1.0000
1,-1.0,-1.0,-1.0,-0.764611,-0.947118,-0.980483,-0.993121,-0.997850,-0.9750
2,-1.0,-1.0,-1.0,-0.765736,-0.945063,-0.979668,-0.992865,-0.997776,-0.9500
3,-1.0,-1.0,-1.0,-0.767146,-0.943088,-0.978833,-0.992599,-0.997699,-0.9250
4,-1.0,-1.0,-1.0,-0.768813,-0.941202,-0.977979,-0.992325,-0.997618,-0.9000
...,...,...,...,...,...,...,...,...,...
1184113,1.0,1.0,1.0,0.530999,0.586766,0.663122,0.735781,0.842895,0.1250
1184114,1.0,1.0,1.0,0.500986,0.559694,0.639927,0.715752,0.826915,0.2500
1184115,1.0,1.0,1.0,0.473012,0.534416,0.618196,0.696872,0.811610,0.3750
1184116,1.0,1.0,1.0,0.410439,0.477706,0.569213,0.653949,0.776051,0.6875


In [19]:
# print statistic data for inference transforms:
for column in df.columns:
    col_data = df[column].to_numpy()

    if column in logarithmic_columns or column in outputs:
        col_data = np.log(col_data) # first transform - log

    col_mean = np.mean(col_data)
    col_std = np.std(col_data)

    print(f"{column}: mean={col_mean} std={col_std}")
    col_data = (col_data - col_mean) / col_std
    print(f"{column}: min={np.min(col_data)} max={np.max(col_data)}")

ro_well: mean=-1.7269386497605308 std=1.7269388050893328
ro_well: min=-1.6666667792427492 max=1.666667010837201
ro_formation: mean=3.4538777271899646 std=3.389313407249595
ro_formation: min=-1.6984156165290543 max=1.6984155647788115
kanisotrop: mean=2.183226068969074 std=1.276033218172284
kanisotrop: min=-0.9272690178582171 max=2.207445614202356
A04M01N: mean=1.1696066630906665 std=1.8981222952922183
A04M01N: min=-2.2750642972209936 max=2.835495775774292
A10M01N: mean=2.1549688040161925 std=2.255211124359088
A10M01N: min=-2.0179301353187635 max=2.636742904204633
A20M05N: mean=2.8295580213737046 std=2.6552511938592653
A20M05N: min=-1.953984183017199 max=2.4430786527659194
A40M05N: mean=3.2474607202004897 std=3.0011638205846554
A40M05N: min=-1.8572676912982564 max=2.289199537124314
A80M10N: mean=3.547436036617802 std=3.3244960293369976
A80M10N: min=-1.7621519117873663 max=2.105355682717262
rad_well: mean=0.08936170212765956 std=0.03429870385983213
rad_well: min=-1.4391710639966195 max=3.

In [20]:
def print_inference_statistic(attributes, df_):
    means = []
    stds = []
    mins = []
    maxes = []

    for column in attributes:
        col_data = df_[column].to_numpy()

        if column in logarithmic_columns or column in outputs:
            col_data = np.log(col_data) # first transform - log

        col_mean = np.mean(col_data)
        col_std = np.std(col_data)

        means.append(col_mean)
        stds.append(col_std)

        col_data = (col_data - col_mean) / col_std

        mins.append(np.min(col_data))
        maxes.append(np.max(col_data))

    print(f"means={means}")
    print(f"stds={stds}")
    print(f"mins={mins}")
    print(f"maxes={maxes}")

In [21]:
print_inference_statistic(inputs, df)

means=[-1.7269386497605308, 3.4538777271899646, 0.08936170212765956, 2.183226068969074]
stds=[1.7269388050893328, 3.389313407249595, 0.03429870385983213, 1.276033218172284]
mins=[-1.6666667792427492, -1.6984156165290543, -1.4391710639966195, -0.9272690178582171]
maxes=[1.666667010837201, 1.6984155647788115, 3.225728246888976, 2.207445614202356]


In [22]:
print_inference_statistic(outputs, df)

means=[1.1696066630906665, 2.1549688040161925, 2.8295580213737046, 3.2474607202004897, 3.547436036617802]
stds=[1.8981222952922183, 2.255211124359088, 2.6552511938592653, 3.0011638205846554, 3.3244960293369976]
mins=[-2.2750642972209936, -2.0179301353187635, -1.953984183017199, -1.8572676912982564, -1.7621519117873663]
maxes=[2.835495775774292, 2.636742904204633, 2.4430786527659194, 2.289199537124314, 2.105355682717262]


### Build Datasets and create dataloaders

In [19]:
class SimpleDataset(Dataset):
    def __init__(self, df_, inputs, outputs, device):
        self.df = df_
        self.inputs = torch.from_numpy(df_[inputs].to_numpy()).float().to(device)
        self.outputs = torch.from_numpy(df_[outputs].to_numpy()).float().to(device)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        item, label = self.inputs[idx], self.outputs[idx]

        return item, label


In [20]:
batch_size = 1000

train_df, test_df = train_test_split(df_transformed, shuffle=True, test_size=0.3)

train_dataset = SimpleDataset(train_df, inputs, outputs, device)
test_dataset = SimpleDataset(test_df, inputs, outputs, device)
full_dataset = SimpleDataset(df_transformed, inputs, outputs, device)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
full_dataset_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)

### Build models

In [21]:
class WeightedMAE(nn.Module):
    def __init__(self, weights):
        super(WeightedMAE, self).__init__()
        self.mae = nn.L1Loss()
        self.weights = weights

    def forward(self, inputs, targets):
        weighted_inputs = inputs * self.weights

        return self.mae(weighted_inputs, targets)

    def to(self, device):
        super().to(device)
        self.weights = self.weights.to(device)


class LinearModel(nn.Module):
    def __init__(self, layers_dims, act_str_list, output_dim):
        super().__init__()
        layers_count = len(layers_dims)
        assert layers_count > 0

        module_list = []
        for i in range(layers_count - 1):
            module_list.append(nn.Linear(layers_dims[i], layers_dims[i + 1]))
        module_list.append(nn.Linear(layers_dims[layers_count - 1], output_dim))

        activations_list = []
        for i in range(layers_count):
            activations_list.append(activations[act_str_list[i]])

        self.linears = nn.ModuleList(module_list)
        self.activations = nn.ModuleList(activations_list)

    def forward(self, x):
        y = x

        for lin, act in zip(self.linears, self.activations):
            y = lin(y)
            y = act(y)

        return y


class LinearLNormModel(nn.Module):
    def __init__(self, layers_dims, act_str_list, output_dim):
        super().__init__()
        layers_count = len(layers_dims)
        assert layers_count > 0

        linears_list = []
        layers_norm_list = []

        for i in range(layers_count - 1):
            in_features, out_features = layers_dims[i], layers_dims[i + 1]
            linears_list.append(nn.Linear(in_features, out_features))
            layers_norm_list.append(nn.LayerNorm(out_features))
        # add last layer
        linears_list.append(nn.Linear(layers_dims[layers_count - 1], output_dim))
        layers_norm_list.append(nn.LayerNorm(output_dim))

        self.linears = nn.ModuleList(linears_list)
        self.activations = nn.ModuleList([activations[act_str_list[i]] for i in range(len(act_str_list))])
        self.layer_normalizations = nn.ModuleList(layers_norm_list)

    def forward(self, x):
        y = x

        for lin, act, norm in zip(self.linears, self.activations, self.layer_normalizations):
            y = lin(y)
            y = norm(y)
            y = act(y)

        return y


# add batch normalization
class LinearBNormModel(nn.Module):
    def __init__(self, layers_dims, act_str_list, output_dim):
        super().__init__()
        layers_count = len(layers_dims)
        assert layers_count > 0

        linears_list = []
        batch_norm_list = []

        for i in range(layers_count - 1):
            in_features, out_features = layers_dims[i], layers_dims[i + 1]
            linears_list.append(nn.Linear(in_features, out_features))
            batch_norm_list.append(nn.BatchNorm1d(out_features))

        linears_list.append(nn.Linear(layers_dims[layers_count - 1], output_dim))
        batch_norm_list.append(nn.BatchNorm1d(output_dim))

        activations_list = []
        for i in range(layers_count):
            activations_list.append(activations[act_str_list[i]])

        self.linears = nn.ModuleList(linears_list)
        self.activations = nn.ModuleList(activations_list)
        self.batch_normalizations = nn.ModuleList(batch_norm_list)

    def forward(self, x):
        y = x

        for lin, act, norm in zip(self.linears, self.activations, self.batch_normalizations):
            y = lin(y)
            y = norm(y)
            y = act(y)

        return y


### Train model

In [49]:
layers_dims = [len(inputs), 40, 120, 1200, 120, 50]
layers_count = len(layers_dims)
activations_string_list = ['leaky-relu' for i in range(layers_count)]
#activations_string_list[-1] = 'sigmoid'

linear_model = LinearModel(layers_dims, activations_string_list, len(outputs)).to(device)
#linear_bn_model = LinearBNormModel(layers_dims, activations_string_list, len(outputs)).to(device)
#linear_ln_model = LinearLNormModel(layers_dims, activations_string_list, len(outputs)).to(device)

model = linear_model
model_name = "linear_model"
linear_model

LinearModel(
  (linears): ModuleList(
    (0): Linear(in_features=4, out_features=40, bias=True)
    (1): Linear(in_features=40, out_features=120, bias=True)
    (2): Linear(in_features=120, out_features=1200, bias=True)
    (3): Linear(in_features=1200, out_features=120, bias=True)
    (4): Linear(in_features=120, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=5, bias=True)
  )
  (activations): ModuleList(
    (0-5): 6 x LeakyReLU(negative_slope=0.01)
  )
)

In [50]:
learning_rate = 0.0001
epoch_count = 1500

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

#loss_function = WeightedMAE(torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0], dtype=float))
loss_function = nn.L1Loss()

In [51]:
epoch_validation = True
train_loss_threshold = 0.0003

train_loss_list, validation_loss_list = train_model(epoch_count, model, optimizer, loss_function, train_loader, test_loader, True, train_loss_threshold)
plot_loss(train_loss_list, "train loss")

Epoch: 0; train loss=0.248373; validation loss=0.085334
Epoch: 1; train loss=0.056940; validation loss=0.051239
Epoch: 2; train loss=0.049298; validation loss=0.046200
Epoch: 3; train loss=0.037637; validation loss=0.025714
Epoch: 4; train loss=0.020479; validation loss=0.017020
Epoch: 5; train loss=0.015683; validation loss=0.014307
Epoch: 6; train loss=0.014397; validation loss=0.013656
Epoch: 7; train loss=0.013252; validation loss=0.012904
Epoch: 8; train loss=0.012287; validation loss=0.012338
Epoch: 9; train loss=0.011650; validation loss=0.011171
Epoch: 10; train loss=0.010885; validation loss=0.011375
Epoch: 11; train loss=0.010172; validation loss=0.010646
Epoch: 12; train loss=0.009418; validation loss=0.011976
Epoch: 13; train loss=0.009227; validation loss=0.008275
Epoch: 14; train loss=0.008718; validation loss=0.008296
Epoch: 15; train loss=0.008375; validation loss=0.007806
Epoch: 16; train loss=0.008171; validation loss=0.007232
Epoch: 17; train loss=0.007664; validatio

In [52]:
test_loss = test_loop(test_loader, model, loss_function)
print(f"test loss={test_loss}")

test loss=0.0006121573153833514


In [53]:
plot_loss(validation_loss_list, "test loss")

### Plot predictions

In [24]:
for _, (X, y) in enumerate(train_loader):
    print(model(X))
    break

tensor([[ 0.0283,  0.0467,  0.0514, -0.0150, -0.1277],
        [ 0.3988,  0.2750,  0.1265, -0.0010, -0.0572],
        [ 0.0397,  0.1307,  0.2489,  0.3416,  0.4503],
        ...,
        [-0.5458, -0.4631, -0.3697, -0.3541, -0.4000],
        [-0.5653, -0.8314, -0.8675, -0.8829, -0.8901],
        [-0.3139, -0.5173, -0.6135, -0.6564, -0.6743]], device='cuda:0',
       grad_fn=<LeakyReluBackward0>)


In [22]:
# create dataloader without shuffle
full_inference_dataset_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=False)

In [56]:
# plot_predictions(outputs, full_dataset_loader, linear_model)

In [57]:
#plot_actual_predictions(outputs, full_inference_dataset_loader, linear_model, attributes_transform_dict, df)

In [23]:
model = torch.load("saved_models/" + "linear_model0_0006122.pth")
model.to(device)

LinearModel(
  (linears): ModuleList(
    (0): Linear(in_features=4, out_features=40, bias=True)
    (1): Linear(in_features=40, out_features=120, bias=True)
    (2): Linear(in_features=120, out_features=1200, bias=True)
    (3): Linear(in_features=1200, out_features=120, bias=True)
    (4): Linear(in_features=120, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=5, bias=True)
  )
  (activations): ModuleList(
    (0-5): 6 x LeakyReLU(negative_slope=0.01)
  )
)

In [19]:
plot_relative_errors(outputs, full_inference_dataset_loader, model, attributes_transform_dict,
                     df, 0.01, device, plots_dir, mode='default+hist', bin_count=100)

#### check predictions manually

In [59]:
predictor = Predictor(full_inference_dataset_loader, df, attributes_transform_dict, model, inputs, outputs)
predictions_dict, actuals_dict = predictor.predict(device)

In [60]:
def compare_prediction(idx: int, prediction_dict, actuals_dict, attribute):
    predicted = prediction_dict[attribute][idx]
    actual = actuals_dict[attribute][idx]
    relative_error = abs(actual - predicted) / actual
    print(f"{idx}: predicted={predicted}; actual={actual}; relative error={relative_error}")

In [61]:
compare_prediction(119896, predictions_dict, actuals_dict, 'A10M01N')

119896: predicted=0.8281782269477844; actual=0.839794; relative error=0.01383167039602995


In [26]:
model(torch.tensor([-1.0, -1, -1, -1]).to(device))

tensor([-0.7639, -0.9476, -0.9801, -0.9923, -0.9970], device='cuda:0',
       grad_fn=<LeakyReluBackward0>)

In [42]:
model(torch.tensor([-1.0,-1.0,-0.952277, -0.575]).to(device))

tensor([-0.7213, -0.9231, -0.9744, -0.9911, -0.9968], device='cuda:0',
       grad_fn=<LeakyReluBackward0>)

### Save model

In [62]:
model.to(cpu)    # attach model to cpu before scripting and saving to prevent cuda meta information saved
scripted_model = torch.jit.script(model)
model_file_name = "saved_models/" + model_name + str(round(test_loss, 7)).replace('.', '_')

scripted_model.save(model_file_name + ".pt") # save torch script model which compatible with pytorch c++ api
torch.save(model, model_file_name + ".pth")   # save model in python services specific format

# attach model back to device:
model.to(device)

LinearModel(
  (linears): ModuleList(
    (0): Linear(in_features=4, out_features=40, bias=True)
    (1): Linear(in_features=40, out_features=120, bias=True)
    (2): Linear(in_features=120, out_features=1200, bias=True)
    (3): Linear(in_features=1200, out_features=120, bias=True)
    (4): Linear(in_features=120, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=5, bias=True)
  )
  (activations): ModuleList(
    (0-5): 6 x LeakyReLU(negative_slope=0.01)
  )
)

In [63]:
#scripted_model(torch.tensor([0.6, 0.362372, 0.04]))

In [64]:
#model(torch.tensor([0.6, 0.362372, 0.04], device=device))